In [4]:
import pandas as pd
import json
import joblib


import json
import tiktoken # for token counting
import numpy as np
from collections import defaultdict

In [5]:
df = joblib.load("train_df.joblib")

In [6]:
df["true_labels"] = df.apply(
    lambda x: {k: v for k, v in zip(x["category"], x["polarity"])}, axis=1
)

In [7]:
df.head()

,text,category,polarity,joint,category_labels,polarity_labels,joint_labels,true_labels
0,staff horrible us,[service],[negative],[service#negative],"[0, 0, 0, 0, 1]","[1, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",{'service': 'negative'}
1,completely fair redeeming factor food average ...,"[food, other]","[positive, negative]","[food#positive, other#negative]","[0, 1, 1, 0, 0]","[1, 0, 1]","[0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]","{'food': 'positive', 'other': 'negative'}"
2,food uniformly exceptional capable kitchen pro...,[food],[positive],[food#positive],"[0, 1, 0, 0, 0]","[0, 0, 1]","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",{'food': 'positive'}
3,gabriela personaly greets recommends eat,[service],[positive],[service#positive],"[0, 0, 0, 0, 1]","[0, 0, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",{'service': 'positive'}
4,go enjoy say get,[other],[positive],[other#positive],"[0, 0, 1, 0, 0]","[0, 0, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]",{'other': 'positive'}


In [11]:
df = df.sample(100).reset_index()

In [12]:
aspects = ["food", "service", "ambience", "price", "other"]
sentiments = ["positive", "neutral", "negative"]

In [13]:
lines = []

for _, row in df.iterrows():
    text = row["text"]
    answer = json.dumps(row["true_labels"])

    lines.append({"messages" : [{"role": "system", "content": f"Your task is to analyse and perform aspect based sentiment analysis on the given restaurant reviews text. The aspects should only be {aspects} and sentiments should only be {sentiments}. Output should only contain mentioned aspects and their respective sentiments as a dictionary. Each aspects should only have one sentiment and not every aspect is necessarily present. Do not provide any further explanation."}, {"role": "user", "content": text}, {"role": "assistant", "content": answer}]})
    


In [14]:
import json

with open('openai.jsonl', 'w') as outfile:
    for entry in lines:
        json.dump(entry, outfile)
        outfile.write('\n')

## Data validation

In [15]:
data_path = "openai.jsonl"

# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

Num examples: 100
First example:
{'role': 'system', 'content': "Your task is to analyse and perform aspect based sentiment analysis on the given restaurant reviews text. The aspects should only be ['food', 'service', 'ambience', 'price', 'other'] and sentiments should only be ['positive', 'neutral', 'negative']. Output should only contain mentioned aspects and their respective sentiments as a dictionary. Each aspects should only have one sentiment and not every aspect is necessarily present. Do not provide any further explanation."}
{'role': 'user', 'content': 'several times totally smitten'}
{'role': 'assistant', 'content': '{"other": "positive"}'}


In [16]:
# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue
        
    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue
        
    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1
        
        if any(k not in ("role", "content", "name", "function_call", "weight") for k in message):
            format_errors["message_unrecognized_key"] += 1
        
        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1
        
        content = message.get("content", None)
        function_call = message.get("function_call", None)
        
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1
    
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


In [17]:
encoding = tiktoken.get_encoding("cl100k_base")

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

In [18]:
# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))
    
print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 4096 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning")

Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 111, 140
mean / median: 119.26, 118.0
p5 / p95: 113.0, 127.10000000000001

#### Distribution of num_assistant_tokens_per_example:
min / max: 6, 25
mean / median: 7.73, 6.0
p5 / p95: 6.0, 13.0

0 examples may be over the 4096 token limit, they will be truncated during fine-tuning


In [19]:
# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 4096

TARGET_EPOCHS = 3
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

Dataset has ~11926 tokens that will be charged for during training
By default, you'll train for 3 epochs on this dataset
By default, you'll be charged for ~35778 tokens
